In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing

In [40]:
def make_mlp(input_size, hidden_size, output_size, hidden_layers=2):
    """
    논문에서 사용하는 표준 MLP: Linear -> ReLU -> Linear ... -> LayerNorm (마지막 제외)
    """
    layers = []
    layers.append(nn.Linear(input_size, hidden_size))
    layers.append(nn.ReLU())
    
    for _ in range(hidden_layers):
        layers.append(nn.Linear(hidden_size, hidden_size))
        layers.append(nn.ReLU())
        
    layers.append(nn.Linear(hidden_size, output_size))
    # GNS는 마지막에 LayerNorm을 쓰지 않는 경우가 많지만, 
    # 안정성을 위해 중간 레이어에는 넣기도 합니다. 여기선 심플하게 갑니다.
    return nn.Sequential(*layers)

In [41]:
import torch

def naive_radius_graph(x, r, batch=None, loop=False):
    """
    torch-cluster 라이브러리 없이 순수 PyTorch로 구현한 radius_graph.
    M1 Mac (MPS)에서도 잘 돌아갑니다.
    """
    # 배치가 없으면 모두 같은 배치로 취급
    if batch is None:
        batch = x.new_zeros(x.size(0), dtype=torch.long)
    
    edge_index_list = []
    
    # 배치별로 따로 계산 (이렇게 해야 다른 시뮬레이션 입자와 연결 안 됨)
    # unique_batches는 보통 [0, 1] 등이 나옵니다.
    for b in torch.unique(batch):
        # 현재 배치(b)에 해당하는 입자들의 마스크(True/False)
        mask = (batch == b)
        
        # 현재 배치에 속한 입자들의 전역 인덱스 (Global Index)
        # 예: 2번째 배치의 입자가 전체 중 356번째~710번째라면 그 번호들
        global_indices = torch.nonzero(mask, as_tuple=True)[0]
        
        # 해당 배치의 입자 위치 데이터 추출
        x_batch = x[mask]
        
        # 거리 계산 (N_b x N_b 행렬)
        # cdist는 MPS(GPU) 가속이 잘 됩니다.
        dist_mat = torch.cdist(x_batch, x_batch)
        
        # 반경 r 이내인 것들 찾기 (True/False 행렬)
        adj_mask = dist_mat < r
        
        # 자기 자신(대각선) 제외
        if not loop:
            adj_mask.fill_diagonal_(False)
            
        # 연결된 인덱스 (Local Index: 0번부터 시작) 추출
        src_local, dst_local = adj_mask.nonzero(as_tuple=True)
        
        # Local Index -> Global Index 변환
        src_global = global_indices[src_local]
        dst_global = global_indices[dst_local]
        
        # 결과 저장
        edge_sub = torch.stack([src_global, dst_global], dim=0)
        edge_index_list.append(edge_sub)
    
    # 아무것도 연결 안 됐을 경우 예외 처리
    if len(edge_index_list) == 0:
        return torch.empty((2, 0), dtype=torch.long, device=x.device)
        
    # 리스트에 모인 엣지들을 하나로 합치기
    return torch.cat(edge_index_list, dim=1)

In [42]:
class GNBlock(MessagePassing):
    def __init__(self, hidden_size):
        super().__init__(aggr='add') # 이웃들의 메시지를 '더해서(add)' 받음
        
        # 엣지 모델: (Node + Node + Edge) -> New Edge
        # 입력: sender_node(128) + receiver_node(128) + current_edge(128) = 384
        self.edge_mlp = make_mlp(hidden_size * 3, hidden_size, hidden_size)
        
        # 노드 모델: (Node + Aggregated Edge) -> New Node
        # 입력: current_node(128) + aggregated_edge(128) = 256
        self.node_mlp = make_mlp(hidden_size * 2, hidden_size, hidden_size)

    def forward(self, x, edge_index, edge_attr):
        # 1. Edge Update & Message Passing
        # propagate 함수가 내부적으로 message() -> aggregate() -> update()를 호출함
        
        # x: [N, hidden] (노드 특징)
        # edge_index: [2, E] (연결 정보)
        # edge_attr: [E, hidden] (엣지 특징)
        
        # update_edge_attr을 리턴받기 위해 message 함수를 직접 호출하는 대신
        # 수동으로 계산하고 propagate를 돌립니다.
        
        # (src, dst) 가져오기
        src, dst = edge_index
        
        # 엣지 업데이트: MLP(Node_i, Node_j, Edge_ij)
        out_edge = torch.cat([x[src], x[dst], edge_attr], dim=1)
        updated_edge_attr = self.edge_mlp(out_edge) # [E, 128]
        
        # 2. Node Update (Aggregation)
        # updated_edge_attr을 메시지로 보내서 노드별로 합침
        aggr_out = self.propagate(edge_index, x=x, edge_attr=updated_edge_attr, size=(x.size(0), x.size(0)))
        
        # 3. Residual Connection (매우 중요!)
        # 기존 엣지 특성에 잔차 연결
        new_edge_attr = edge_attr + updated_edge_attr
        
        # 기존 노드 특성에 잔차 연결 (propagate 결과인 aggr_out을 입력으로 사용)
        new_x = x + aggr_out
        
        return new_x, new_edge_attr

    def message(self, edge_attr):
        # propagate가 호출할 때 이 값을 집계(aggr='add') 함
        return edge_attr

    def update(self, aggr_out, x):
        # 집계된 메시지(aggr_out)와 내 원래 상태(x)를 합쳐서 업데이트
        input_vec = torch.cat([x, aggr_out], dim=1)
        return self.node_mlp(input_vec)

In [43]:
class GNS(nn.Module):
    def __init__(self, input_dim=2, hidden_size=128, num_layers=10, radius=0.015):
        super().__init__()
        self.radius = radius
        self.input_dim = input_dim # 2D(x,y) or 3D(x,y,z)
        
        # ---------------------------
        # 1. ENCODER
        # ---------------------------
        # 입자 타입 임베딩 (총 9가지 타입이라 가정)
        self.type_embedding = nn.Embedding(9, 16) 
        
        # Node Encoder: 
        # 입력: 과거 5개 속도(input_dim * 5) + 벽면 거리 등(선택) + 타입임베딩(16)
        # Window가 6이면 -> 속도는 5개 나옴
        node_input_size = (input_dim * 5) + 16 
        self.node_encoder = make_mlp(node_input_size, hidden_size, hidden_size)
        
        # Edge Encoder:
        # 입력: 상대 위치(input_dim) + 거리(1) = 3 (2D 기준)
        self.edge_encoder = make_mlp(input_dim + 1, hidden_size, hidden_size)
        
        # ---------------------------
        # 2. PROCESSOR
        # ---------------------------
        self.gn_layers = nn.ModuleList(
            [GNBlock(hidden_size) for _ in range(num_layers)]
        )
        
        # ---------------------------
        # 3. DECODER
        # ---------------------------
        # Latent(128) -> 가속도(input_dim)
        self.decoder = make_mlp(hidden_size, hidden_size, input_dim, hidden_layers=2)


    def forward(self, batch_data):
        # batch_data는 PyG DataLoader가 준 Batch 객체입니다.
        # x: (Total_N, 6, Dim) -> 위치 시퀀스
        # particle_type: (Total_N,)
        # batch: (Total_N,) -> 배치 인덱스
        
        pos_seq = batch_data.x       # 위치 정보
        p_type = batch_data.particle_type
        batch_idx = batch_data.batch
        
        # --- [전처리] 위치 시퀀스를 속도(Velocity)로 변환 ---
        # 논문에서는 절대 위치 대신 "최근 5개 속도"를 입력으로 씁니다.
        # pos_seq: (N, 6, D)
        # velocities: (N, 5, D) -> t1-t0, t2-t1, ...
        velocities = pos_seq[:, 1:] - pos_seq[:, :-1]
        
        # Flatten: (N, 5*D) -> MLP에 넣기 좋게 핌
        node_features = velocities.reshape(velocities.size(0), -1)
        
        # --- [Encoder] 1. Node Embedding ---
        # 입자 타입 임베딩
        type_emb = self.type_embedding(p_type) # (N, 16)
        
        # 속도 정보 + 타입 정보 결합
        node_input = torch.cat([node_features, type_emb], dim=1)
        x = self.node_encoder(node_input) # (N, 128) -> Latent State
        
        
        # --- [Encoder] 2. Graph Construction (Dynamic!) ---
        # 가장 최근 위치(pos_seq[:, -1])를 기준으로 반경(radius) 내 이웃 연결
        curr_pos = pos_seq[:, -1]
        
        # radius_graph가 배치 정보를 보고, 같은 시뮬레이션 안에서만 연결해줍니다!
        edge_index = naive_radius_graph(curr_pos, r=self.radius, batch=batch_idx, loop=False)
        
        # --- [Encoder] 3. Edge Embedding ---
        # sender(j) -> receiver(i)
        src, dst = edge_index
        
        # 상대 위치 (Relative Position): pos_j - pos_i
        rel_pos = curr_pos[src] - curr_pos[dst]
        # 거리 (Distance): ||pos_j - pos_i||
        edge_dist = torch.norm(rel_pos, dim=-1, keepdim=True)
        
        # 엣지 피처: [상대위치, 거리]
        edge_input = torch.cat([rel_pos, edge_dist], dim=1)
        edge_attr = self.edge_encoder(edge_input) # (E, 128)
        
        
        # --- [Processor] Message Passing Loop ---
        for gn_layer in self.gn_layers:
            x, edge_attr = gn_layer(x, edge_index, edge_attr)
            
            
        # --- [Decoder] Predict Acceleration ---
        pred_acc = self.decoder(x) # (N, Dim)
        
        return pred_acc

In [44]:
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from dataset import GNSDataset

In [45]:
# -----------------------------------------
# 1. 설정 (Configuration)
# -----------------------------------------
# M1 Mac은 'mps'를 쓰면 빠르지만, 가끔 호환성 문제가 있으면 'cpu'로 바꾸세요.
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

BATCH_SIZE = 2
LR = 1e-4
NUM_EPOCHS = 2
DATA_DIR = "./datasets/WaterDropSample/train" # 경로 확인 필수!

Using device: mps


In [46]:
# -----------------------------------------
# 2. 데이터 & 모델 준비
# -----------------------------------------
# 데이터셋 (아까 만든 Virtual Indexing 버전)
train_dataset = GNSDataset(data_dir=DATA_DIR, window_length=7, mode='train')

# 로더 (PyG 로더 사용!)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=0 # M1 에러 방지용 0
)

# 모델 초기화
model = GNS(input_dim=2, hidden_size=128, num_layers=10, radius=0.015).to(device)

# 최적화 도구
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

[TRAIN] Scanning 2 files for indexing...
[TRAIN] Ready! Total Samples: 1990


In [ ]:
# -----------------------------------------
# 3. 학습 루프 (Training Loop)
# -----------------------------------------
print("🚀 학습 시작!")
model.train()

for epoch in range(NUM_EPOCHS):
    total_loss = 0
    num_batches = 0
    
    for batch_idx, batch in enumerate(train_loader):
        batch = batch.to(device)
        
        # --- [중요] 타겟 가속도(Ground Truth Acceleration) 계산 ---
        # 데이터셋은 '다음 위치(y)'를 줍니다. 
        # 하지만 우리는 '가속도'를 맞춰야 하므로 변환합니다.
        # 가속도 = 다음위치 - 현재위치 - 현재속도
        # a_t = p_{t+1} - p_t - v_t
        #     = p_{t+1} - 2*p_t + p_{t-1}
        
        next_pos = batch.y                # p_{t+1} (Target)
        curr_pos = batch.x[:, -1]         # p_t (Current)
        prev_pos = batch.x[:, -2]         # p_{t-1} (Previous)
        
        # 정답 가속도 계산
        # (주의: 노이즈가 섞인 입력 기준으로 가속도를 계산해야 모델이 노이즈 보정을 배웁니다)
        current_vel = curr_pos - prev_pos
        target_acc = next_pos - curr_pos - current_vel
        
        # --- Forward & Backward ---
        optimizer.zero_grad()
        
        # 모델 예측 (pred_acc)
        pred_acc = model(batch)
        
        # Loss 계산 (가속도 끼리 비교)
        loss = F.mse_loss(pred_acc, target_acc)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1} | Batch {batch_idx} | Loss: {loss.item():.6f}")

    avg_loss = total_loss / num_batches
    print(f"=== Epoch {epoch+1} Done. Avg Loss: {avg_loss:.6f} ===\n")

print("🎉 학습 완료!")

In [51]:
# -----------------------------------------
# 4. 모델 저장
# -----------------------------------------
torch.save(model.state_dict(), "gns_model_water.pth")
print("모델 저장 완료: gns_model_water.pth")

모델 저장 완료: gns_model_water.pth


In [58]:
import torch
import numpy as np
from torch_geometric.data import Data, Batch

def rollout(model, data, num_steps, device):
    """
    model: 학습된 GNS 모델
    data: 초기 상태가 포함된 PyG Data 객체 (dataset[0])
    num_steps: 시뮬레이션할 총 스텝 수 (예: 600)
    """
    model.eval()
    
    # 초기 윈도우(6개) 가져오기
    # data.x shape: (6, N, 2) -> (N, 6, 2)로 변환 필요할 수도 있음 (Dataset 구현에 따라 다름)
    # 여기서는 Dataset이 (N, 6, 2)를 준다고 가정 (아까 그렇게 바꿨으니까요!)
    current_pos_seq = data.x.to(device) # (N, 6, 2)
    particle_type = data.particle_type.to(device)
    
    # 결과를 저장할 리스트 (초기 6개 포함)
    trajectory = [current_pos_seq[:, i, :].cpu().numpy() for i in range(current_pos_seq.size(1))]
    
    # 마지막 위치와 속도 준비
    curr_pos = current_pos_seq[:, -1, :] # p_t
    prev_pos = current_pos_seq[:, -2, :] # p_{t-1}
    curr_vel = curr_pos - prev_pos       # v_t
    
    print(f"🚀 Rollout Start! (Total {num_steps} steps)")
    
    with torch.no_grad():
        for step in range(num_steps):
            # 1. 모델 입력용 Batch 만들기
            # Rollout은 1개 샘플만 하므로 batch 벡터는 모두 0
            batch_idx = torch.zeros(curr_pos.size(0), dtype=torch.long, device=device)
            
            # 현재 시퀀스 상태로 Data 객체 생성
            batch_data = Batch(
                x=current_pos_seq, 
                particle_type=particle_type, 
                batch=batch_idx
            )
            
            # 2. 가속도 예측
            pred_acc = model(batch_data) # (N, 2)
            
            # 3. 오일러 적분 (Euler Integration) -> 다음 위치 계산
            # v_{t+1} = v_t + a_pred
            # p_{t+1} = p_t + v_{t+1}
            next_vel = curr_vel + pred_acc
            next_pos = curr_pos + next_vel
            
            # 4. 결과 저장
            trajectory.append(next_pos.cpu().numpy())
            
            # 5. 다음 스텝 준비 (Sliding Window)
            # 현재 시퀀스에서 맨 앞(오래된 것) 버리고, 맨 뒤에 새 위치 추가
            # (N, 6, 2) -> (N, 5, 2) -> (N, 6, 2)
            current_pos_seq = torch.cat([current_pos_seq[:, 1:, :], next_pos.unsqueeze(1)], dim=1)
            
            # 변수 업데이트
            curr_pos = next_pos
            curr_vel = next_vel
            
            if step % 50 == 0:
                print(f"Generating step {step}/{num_steps}...")
                
    # 리스트를 Numpy 배열로 변환 (Time, N, Dim)
    return np.stack(trajectory, axis=0)

In [59]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# 기존 코드 수정 버전: 데이터를 인자로 직접 받음
def visualize_rollout(pred_pos_data, type_data, skip_step=5):
    """
    pred_pos_data: (Time, N, 2) 형태의 Numpy 배열 (Rollout 결과)
    type_data: (N,) 형태의 입자 타입
    """
    num_steps = pred_pos_data.shape[0]
    
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(0, 1) 
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.set_title(f"GNS Rollout Prediction")
    
    color_map = {
        3: "black",  # Boundary
        0: "green",  # Solid
        7: "magenta",# Goop
        6: "gold",   # Sand
        5: "blue",   # Water
    }
    # type_data가 Tensor일 수 있으니 numpy로 변환 안전장치
    if isinstance(type_data, torch.Tensor):
        type_data = type_data.cpu().numpy()
        
    colors = [color_map.get(t, 'gray') for t in type_data]
    
    initial_pos = pred_pos_data[0]
    scatter = ax.scatter(initial_pos[:, 0], initial_pos[:, 1], s=15, c=colors, alpha=0.7, edgecolors='none')
    
    def update(frame):
        current_pos = pred_pos_data[frame]
        scatter.set_offsets(current_pos)
        ax.set_title(f"Step: {frame}/{num_steps}")
        return scatter,

    anim = animation.FuncAnimation(
        fig, update, frames=range(0, num_steps, skip_step), interval=30, blit=True
    )
    plt.close()
    return anim

In [61]:
# 1. 테스트할 데이터 하나 가져오기 (Valid 셋이나 Train 셋에서)
# Dataset이 이미 shuffle=True라면 매번 다른 게 나오겠지만, 테스트용으로 0번 인덱스 가져옴
model = GNS()
model.load_state_dict(torch.load('./models/WaterDropSample/gns_model_1_995.pth'))
model.to(device)
valid_dataset = GNSDataset(data_dir=DATA_DIR, window_length=7, mode='valid')
test_data = valid_dataset[0] 

# 2. Rollout 수행 (예: 300 스텝만 예측해보기)
pred_trajectory = rollout(model, test_data, num_steps=300, device=device)

# 3. 시각화!
anim = visualize_rollout(pred_trajectory, test_data.particle_type)
anim # 주피터 노트북이면 영상이 나옵니다.

[VALID] Scanning 2 files for indexing...
[VALID] Ready! Total Samples: 1990
🚀 Rollout Start! (Total 300 steps)
Generating step 0/300...
Generating step 50/300...
Generating step 100/300...
Generating step 150/300...
Generating step 200/300...
Generating step 250/300...


In [56]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
import numpy as np

# 주피터 노트북 설정
rc('animation', html='jshtml')

In [50]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Image, display

def save_comparison_gif(ground_truth, prediction, particle_type, filename="comparison.gif", skip_step=5):
    """
    Left: Ground Truth (Real)
    Right: Prediction (GNS)
    """
    print(f"⚖️ 비교 GIF 생성 중: {filename} ...")
    
    # 두 시뮬레이션 중 짧은 쪽에 맞춰서 길이 조정
    num_steps = min(ground_truth.shape[0], prediction.shape[0])
    
    # 그래프 2개 생성 (1행 2열)
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # 공통 설정
    titles = ["Ground Truth (Real)", "GNS Prediction"]
    scatters = []
    
    # 색상맵 설정
    color_map = {3: "black", 0: "green", 7: "magenta", 6: "gold", 5: "blue"}
    if isinstance(particle_type, torch.Tensor):
        particle_type = particle_type.cpu().numpy()
    colors = [color_map.get(t, 'gray') for t in particle_type]
    
    for ax, title, data in zip(axes, titles, [ground_truth, prediction]):
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_aspect('equal')
        ax.grid(True, linestyle='--', alpha=0.3)
        ax.set_title(title)
        
        # 초기 위치 그리기
        pos = data[0]
        scat = ax.scatter(pos[:, 0], pos[:, 1], s=15, c=colors, alpha=0.7, edgecolors='none')
        scatters.append(scat)
        
    def update(frame):
        # Ground Truth 업데이트
        scatters[0].set_offsets(ground_truth[frame])
        # Prediction 업데이트
        scatters[1].set_offsets(prediction[frame])
        
        fig.suptitle(f"Step: {frame}/{num_steps}", fontsize=12)
        return scatters[0], scatters[1]

    # 애니메이션 생성
    anim = animation.FuncAnimation(
        fig, update, frames=range(0, num_steps, skip_step), interval=30, blit=True
    )
    
    anim.save(filename, writer='pillow', fps=30)
    plt.close()
    
    print("✅ 비교 완료!")
    return display(Image(filename=filename))

In [ ]:
# 사용 예시
# Valid 셋의 0번 파일로 테스트
run_gns_evaluation(
    model=model, 
    data_dir="./datasets/WaterDropSample/valid",
    trajectory_idx=0,                                 # 보고 싶은 파일 번호
    device=device,
    filename="result_0.gif"
)